## **Assignment:** Multi-Layer Perceptron using training (4 inputs) and test (2 outputs) data, MLP with 4 inputs, predicting 2 outputs.

In [40]:
# Import
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense

In [41]:
# Getting dataset
train_data = pd.read_csv('./datasets/5_Training.csv')
test_data = pd.read_excel('./datasets/5_Test.xlsx')
train_data.info()
# train_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   A (mm)       1000 non-null   float64
 1   B (mm)       1000 non-null   float64
 2   C (mm3/s)    1000 non-null   float64
 3   D (degrees)  1000 non-null   float64
 4   y1 (?m)      1000 non-null   float64
 5   y2 (m.sec)   1000 non-null   float64
 6   Unnamed: 6   0 non-null      float64
dtypes: float64(7)
memory usage: 54.8 KB


##### Data Preprocessing
Steps:
1. Drop unneeded columns.
2. Rename the input columns to x1,x2,x3,x4.
3. Rename the output columns to y1,y2.
4. Split the training & test data into inputs and outputs.
    - train_data -> train_input, train_output
    - test_data -> test_input, test_output
##### New Things
- `StandardScaler()` & `fit_transform()`:
    - `fit`: Calculate the mean and standard deviation for each feature in the training data.
    - `transform`: Use these calculated statistics to standardize the data. Each feature is centred around 0 and scaled to have a standard deviation of one.
- `train_test_split()`:
    - `test_size=0.2`: 20% of the dataset will be used for testing / validating the model.
    - `random_state=42`: Controls the randomness of the data splitting process. Setting a specific value ensures that this result is reproducible in the future. If set to `None`, it uses the current system time or another source of true randomness, but you lose the reproducibility of the process.

In [42]:
# Training data
train_data.drop(columns=['Unnamed: 6'],inplace=True)
train_data.rename(columns={'A (mm)':'x1',
                           'B (mm)':'x2',
                           'C (mm3/s)':'x3',
                           'D (degrees)':'x4',
                           'y1 (?m)':'y1',
                           'y2 (m.sec)':'y2'},
                  inplace=True)

x = train_data[['x1','x2','x3','x4']]
x_scaler = StandardScaler()
x_scaled = x_scaler.fit_transform(x)
y = train_data[['y1','y2']]
y_scaler = StandardScaler()
y_scaled = y_scaler.fit_transform(y)

test_data.rename(columns={'A (mm)':'x1',
                           'B (mm)':'x2',
                           'C (mm3/s)':'x3',
                           'D (degrees)':'x4',
                           'y1 (μm)':'y1',
                           'y2 (m.sec)':'y2'},
                 inplace=True)

x_test = train_data[['x1','x2','x3','x4']]
y_test = train_data[['y1','y2']]
x_test_scaled = x_scaler.fit_transform(x_test)
y_test_scaled = y_scaler.fit_transform(y_test)
x_train,x_val,y_train,y_val = train_test_split(x_scaled,y_scaled,test_size=0.2,random_state=42)

In [43]:
# Build the model
model = Sequential()
model.add(Dense(4,input_dim=4,activation='sigmoid',name='Input')) # Input Layer
model.add(Dense(4,activation='sigmoid',name='Hidden1')) # Hidden Layer
model.add(Dense(4,activation='sigmoid',name='Hidden2')) # Hidden Layer
model.add(Dense(4,activation='sigmoid',name='Hidden3')) # Hidden Layer
model.add(Dense(4,activation='sigmoid',name='Hidden4')) # Hidden Layer
model.add(Dense(2,activation='linear',name='Output')) # Output Layer

# Compile & Fit
model.compile(loss='mean_squared_error',optimizer='adam',metrics=['mean_squared_error'])
model.fit(x_train,y_train,epochs=250,verbose=0)

In [ ]:
# Evaluate the model
# x_scaler.inverse_transform(x_test_scaled) to inverse the transform operation.
eval = model.evaluate(x_test_scaled,y_test_scaled)
print("Evaluation: ",eval)

32/32 [==============================] - 0s 1ms/step - loss: 0.0041 - mean_squared_error: 0.0041
Evaluation:  [0.004118608310818672, 0.004118608310818672]


In [51]:
# Predict the output
pred_y = y_scaler.inverse_transform(model.predict(x_test_scaled))[:5]
ground_truth = y_test[:5]

print("Predicted values: \n",pred_y)
print()
print("Ground Truth: \n",ground_truth)

Predicted values: 
 [[9.9409361e+00 4.6740953e+05]
 [1.4547093e+01 4.4826647e+05]
 [1.0644514e+01 4.6448256e+05]
 [1.1177522e+01 4.6226569e+05]
 [1.4694247e+01 4.4765572e+05]]

Ground Truth: 
           y1           y2
0   9.859034  467401.2104
1  14.475502  448411.4769
2  10.607495  464842.1834
3  11.053403  462351.5094
4  14.683481  448091.6552
